# Capítulo 07 - Manutenção e Otimização

## 🧹 Objetivo

Tabelas Iceberg acumulam "lixo" (arquivos de dados antigos, snapshots expirados, metadados obsoletos) ao longo do tempo.
Para manter a performance e reduzir custos de storage, precisamos de rotinas de manutenção.

Neste capítulo vamos explorar:
1.  **Expire Snapshots**: Remover versões antigas e liberar espaço em disco.
2.  **Rewrite Data Files (Compaction)**: Compactar muitos arquivos pequenos em poucos arquivos grandes.
3.  **Remove Orphan Files**: Limpar arquivos que não são referenciados por nenhum snapshot.

## 🔧 Requisitos

- Configuração automática incluída neste notebook.
- Se a tabela `default.sales` não existir, será criada com dados de exemplo.
- Se já existir, será validada e complementada se necessário.

In [9]:
import pyiceberg
from pyiceberg.catalog.sql import SqlCatalog
from pyiceberg.schema import Schema
from pyiceberg.types import NestedField, StringType, LongType, DoubleType, TimestampType
import duckdb
import pandas as pd
import pyarrow as pa
import os
import time
from datetime import datetime, timedelta
from pyiceberg.exceptions import NoSuchNamespaceError

# Configuração Paths
WAREHOUSE_PATH = './iceberg_warehouse'
CATALOG_DB = f"{WAREHOUSE_PATH}/catalog.db"

# Garantir diretório
os.makedirs(WAREHOUSE_PATH, exist_ok=True)

# Configurar catálogo
catalog = SqlCatalog(
    "default",
    **{
        "type": "sql",
        "uri": f"sqlite:///{CATALOG_DB}",
        "warehouse": f"file://{os.path.abspath(WAREHOUSE_PATH)}",
    }
)

def setup_environment(catalog):
    """Garante que a tabela e os dados necessários existem para a aula."""
    table_name = "default.sales"
    namespace = "default"

    # Criar namespace se não existir
    try:
        catalog.create_namespace(namespace)
        print(f"✅ Namespace '{namespace}' criado.")
    except Exception:
        pass
    
    # 1. Definir Schema
    schema = Schema(
        NestedField(1, "id", LongType(), required=True),
        NestedField(2, "created_at", TimestampType(), required=True),
        NestedField(3, "product_name", StringType(), required=False),
        NestedField(4, "amount", DoubleType(), required=False),
        NestedField(5, "quantity", LongType(), required=False),
        NestedField(6, "category", StringType(), required=False)
    )

    try:
        try:
            tbl = catalog.load_table(table_name)
            print(f"✅ Tabela existente encontrada: {table_name}")
            
            # Verificar se tem coluna category
            fields = [f.name for f in tbl.schema().fields]
            if "category" not in fields:
                print("⚠️ Coluna 'category' ausente. Adicionando...")
                with tbl.update_schema() as update:
                    update.add_column("category", StringType())
                print("✅ Coluna 'category' adicionada.")

        except Exception:
            print(f"⚠️ Tabela não encontrada. Criando {table_name}...")
            tbl = catalog.create_table(table_name, schema)
            print(f"✅ Tabela criada.")

        # 2. Gerar Histórico (Snapshots)
        print("🔍 Verificando histórico de snapshots...")
        if len(tbl.snapshots()) < 3:
            print("⚠️ Histórico insuficiente. Gerando dados de teste...")
            
            # Schema explícito do PyArrow para garantir compatibilidade (campos required/non-null)
            pa_schema = pa.schema([
                ('id', pa.int64(), False), # False = Not Nullable
                ('created_at', pa.timestamp('us'), False), # False = Not Nullable
                ('product_name', pa.string()),
                ('amount', pa.float64()),
                ('quantity', pa.int64()),
                ('category', pa.string())
            ])
            
            for i in range(3):
                df = pd.DataFrame({
                    'id': range(i*10, i*10 + 5),
                    'created_at': [datetime.now()] * 5,
                    'product_name': [f'Product_{k}' for k in range(5)],
                    'amount': [100.0 + k for k in range(5)],
                    'quantity': [1] * 5,
                    'category': ['Electronics' if k % 2 == 0 else 'Books' for k in range(5)]
                })
                df['created_at'] = df['created_at'].astype('datetime64[us]')
                
                # Usar schema explícito na conversão
                pa_table = pa.Table.from_pandas(df, schema=pa_schema)
                tbl.append(pa_table)
                print(f"   -> Snapshot {i+1} criado.")
                time.sleep(1) 
        
        print(f"✅ Ambiente pronto! Total de snapshots: {len(tbl.snapshots())}")
        return tbl
        
    except Exception as e:
        print(f"❌ Erro fatal no setup: {e}")
        raise e

# Executar Setup
tbl = setup_environment(catalog)
print(f"Schema atual:\n{tbl.schema()}")

✅ Tabela existente encontrada: default.sales
🔍 Verificando histórico de snapshots...
⚠️ Histórico insuficiente. Gerando dados de teste...
   -> Snapshot 1 criado.
   -> Snapshot 2 criado.
   -> Snapshot 3 criado.
✅ Ambiente pronto! Total de snapshots: 3
Schema atual:
table {
  1: id: required long
  2: created_at: required timestamp
  3: product_name: optional string
  4: amount: optional double
  5: quantity: optional long
  6: category: optional string
}


## 1. Analisando Snapshots

Cada operação de escrita (append, overwrite, update schema) cria um novo snapshot.
Vamos listar o histórico atual.

In [17]:
import pandas as pd

def list_snapshots(table):
    snapshots = table.snapshots()
    data = []
    for s in snapshots:
        ts = datetime.fromtimestamp(s.timestamp_ms / 1000)
        data.append({
            "snapshot_id": s.snapshot_id,
            "timestamp": ts,
            "operation": s.summary.get("operation", "unknown"),
            "records": s.summary.get("added-records", "0")
        })
    return pd.DataFrame(data).sort_values("timestamp")

df_snaps = list_snapshots(tbl)
print(f"Total de snapshots: {len(df_snaps)}")
df_snaps

Total de snapshots: 5


,snapshot_id,timestamp,operation,records
0,458005602930949974,2026-01-22 18:50:40.787,Operation.APPEND,5
1,6407515911490158060,2026-01-22 18:50:41.856,Operation.APPEND,5
2,3509319127170459358,2026-01-22 18:50:42.969,Operation.APPEND,5
3,2007133789407081121,2026-01-22 18:51:48.356,Operation.DELETE,None
4,8774728470651936363,2026-01-22 18:51:48.443,Operation.APPEND,15


## 2. Expire Snapshots (Limpando o Histórico)

Para liberar espaço, removemos snapshots antigos que não são mais necessários para Time Travel.
Isso remove a referência no metadata. (Obs: A remoção física dos arquivos órfãos é um passo subsequente, geralmente feito via `remove_orphan_files`, mas aqui focaremos na limpeza do metadata).

Vamos definir uma data de corte. Tudo anterior a essa data será marcado para expiração.

> **Nota**: O PyIceberg executa isso atualizando o arquivo de metadados (`metadata.json`). A exclusão física dos arquivos de dados antigos depende da implementação do Catalog/IO (em S3/Cloud geralmente funciona bem, localmente o suporte varia).

In [ ]:
### ⚠️ Nota sobre Expire Snapshots

Atualmente, o **PyIceberg** fornece suporte robusto para Leitura, Escrita e Evolução de Metadados.
Entretanto, a API de Manutenção (`expire_snapshots`, `remove_orphan_files`) ainda está em desenvolvimento ativo.

Em ambientes de produção integrados, essa limpeza costuma ser feita via procedimentos do **Spark** ou **Trino**:
```sql
-- Exemplo em Spark/Trino
CALL catalog.system.expire_snapshots(table => 'default.sales', older_than => TIMESTAMP '...');
```

O PyIceberg permite gerenciar ramos e tags via `manage_snapshots()`, mas a limpeza final de metadata/arquivos ainda não está exposta no objeto `Table`.
Manteremos este tópico como conceitual por enquanto.

---
Vamos focar no que conseguimos fazer localmente: **Otimizar a estrutura física dos dados (Compaction/Rewrite)**.

Tentando expirar snapshots anteriores a: 2026-01-22 18:50:42.969000
Operação não suportada ou erro: 'Table' object has no attribute 'expire_snapshots'


## 3. Otimização via Particionamento

Uma das maiores otimizações em Big Data é o **Partition Pruning**.
O Iceberg permite evoluir o esquema de particionamento (Partition Evolution) sem reescrever a tabela imediatamente.

Vamos alterar a tabela para ser particionada por mês do campo `timestamp` (se existir) ou por `category`.
Como adicionamos `category` no capítulo anterior, vamos usar esse campo se disponível, ou `quantity` (bucket) se não.
Vou assumir que o campo `category` existe (criado no Cap 06). Caso não exista (se o usuário pulou), usamos outro.

### Verificando Schema
Vamos checar se temos `category` ou uma coluna de data `date_sale`. Temos `timestamp` no dataset original.
No cap 06 adicionamos `category`.
Vamos adicionar uma spec de partição: `identity(category)`.
Isso vai separar fisicamente os arquivos por categoria.

In [13]:
try:
    # Adicionando particionamento por 'category'
    # Se 'category' for null, ficará numa pasta default.
    # Nota: Dados antigos CONTINUAM não particionados até serem reescritos.
    
    # Verificando se campo existe
    schema_fields = [f.name for f in tbl.schema().fields]
    if 'category' in schema_fields:
        print("Adicionando partição por 'category'...")
        with tbl.update_spec() as update:
            update.add_identity("category")
        print("Spec de partição atualizada!")
    else:
        print("Campo 'category' não encontrado. Adicione via Cap 06 ou use outro campo.")
        
    print(f"Nova Spec:\n{tbl.spec()}")

except Exception as e:
    print(f"Erro ao atualizar spec (talvez já particionado?): {e}")

Adicionando partição por 'category'...
Spec de partição atualizada!
Nova Spec:
[
  1000: category: identity(6)
]


## 4. Rewrite Data Files (Compactação e Reorganização)

Como ativamos o particionamento agora, os dados antigos **ainda estão na estrutura antiga** (sem pastas de partição).
Para aplicar a nova estrutura e otimizar o tamanho dos arquivos (Compactação), precisamos reescrever os dados.

Como o PyIceberg (ainda) não possui um motor de compactação distribuído nativo (como Spark), faremos uma **Compactação Lógica** via `overwrite`:
1. Lemos todos os dados atuais.
2. Reescrevemos para a tabela. O writer do PyIceberg detectará a nova `Partition Spec` e organizará os arquivos fisicamente.

> **Cuidado**: Em produção com petabytes, use Spark/Trino procedures (`rewrite_data_files`). Para datasets que cabem na memória/disco local, essa abordagem funciona.

In [15]:
from pyiceberg.expressions import AlwaysTrue

# 1. Ler dados atuais para memória
# (Em produção, faríamos isso em batches ou usaríamos Spark)
arrow_data = tbl.scan().to_arrow()
print(f"Lendo {len(arrow_data)} registros...")

# 2. Executar Overwrite (Substituição Total)
# Isso deleta logicamente os arquivos antigos e escreve novos já organizados pelas partições
if len(arrow_data) > 0:
    print("Reescrevendo dados (Compactação + Particionamento)...")
    tbl.overwrite(arrow_data, overwrite_filter=AlwaysTrue())
    print("Sucesso!")
else:
    print("Tabela vazia.")

Lendo 15 registros...
Reescrevendo dados (Compactação + Particionamento)...
Sucesso!


## 5. Verificação da Estrutura Física

Vamos verificar se os novos arquivos foram escritos nos diretórios de partição (`category=...`).
Faremos isso inspecionando o plano de scan.

In [16]:
scan = tbl.scan()
# plan_files() retorna um generator de FileScanTask
tasks = list(scan.plan_files())

print(f"Total de arquivos de dados: {len(tasks)}")
print("-" * 50)
for i, task in enumerate(tasks):
    # Pegando o caminho relativo para facilitar leitura
    full_path = task.file.file_path
    relative = full_path.split("iceberg_warehouse")[-1] if "iceberg_warehouse" in full_path else full_path
    print(f"Arquivo {i+1}: ...{relative}")
    # Se particionamento funcionou, veremos 'category=Nome' no caminho

# Conclusão
print("\nSe você vê pastas como 'category=...' no caminho, o Particionamento funcionou!")
print(f"Tipo de operação no último snapshot: {tbl.current_snapshot().summary.get('operation')}")

Total de arquivos de dados: 2
--------------------------------------------------
Arquivo 1: .../default/sales/data/category=Electronics/00000-0-bd56202a-7fca-428a-be21-b6004d20e794.parquet
Arquivo 2: .../default/sales/data/category=Books/00000-1-bd56202a-7fca-428a-be21-b6004d20e794.parquet

Se você vê pastas como 'category=...' no caminho, o Particionamento funcionou!
Tipo de operação no último snapshot: Operation.APPEND
